In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from  tensorflow.keras.preprocessing.sequence import pad_sequences


from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import re

In [2]:
data = pd.read_csv('mary_oliver.csv')
poems_mo = data.iloc[:,1]
display(poems_mo)
#poems=poems_mo

0      \nThere is a thing in me that dreamed of trees...
1      \nShe sends me news of blue jays, frost,\nOf s...
2      \nShe steps into the dark swamp\nwhere the lon...
3      \nMy father, for example,\nwho was young once\...
4      \nClosing the book, I find I have left my head...
                             ...                        
96     \nDon't call this world adorable, or useful, t...
97     \nAll night\nI float\nin the shallow ponds\nwh...
98     \nBone\n1.\nUnderstand, I am always trying to ...
99     \nYou do not have to be good.\nYou do not have...
100    \nHow necessary it is to have opinions! I thin...
Name: text, Length: 101, dtype: object

In [3]:
## Add data from Pavlo Neruda
data_pn = pd.read_csv('pablo_neruda.csv')
poems_pn = data_pn.iloc[:,1]
pd.set_option('display.max_rows', 131)
#display(poems_pn)


In [4]:
spanish_poems=[4,37,38,74,79,83]

for i in spanish_poems:
    print (poems_pn[i])
    poems_pn.pop(i)




Era el crepúsculo de la iguana.
Desde la arcoirisada crestería
su leengua como un dardo
se hundía en la verdura,
el hormiguero monacal pisaba
con melodioso pie la selva,
el guanaco fino como el oxigeno
en las anchas alturas pardas
iba calzando botas de oro,
mientras la llama abria cándidos
ojos en la delicadeza
del mundo lleno de rocio.
Los monos trenzaban un hilo
interminablemente erótico
en las riberas de la aurora,
derribando muros de polen
y espantando el vuelo violeta
de las mariposas de Muzo
Era la noche de los caimanes,
la noche pura y pululante
de hocicos saliendo del légamo,
y de las ciénagas soñolientas
un ruido opaco de armaduras
volvía al origen terrestre.
El jaguar tocaba las hojas
con su ausencia fosforescente,
el puma corre en el ramaje
como el fuego devorador
mientras arden en él los ojos
alcohólicos de la selva.
Los tejones rascan los pies
del río, husmean el nido
cuya delicia palpitante
atacarán con dientes rojos.
Y en el fondo del agua magna,
como el círulo de la t

In [ ]:
#poems=poems_pn

In [ ]:
# Add other poems about nature:
data_n = pd.read_csv('nature.csv')
poems_n = data_n.iloc[:,1]
display(poems_n)

In [ ]:
type(poems_mo)

In [5]:
#poems_mo = pd.DataFrame(poems_mo)
#poems_pn = pd.DataFrame(poems_pn)

poems = pd.concat([poems_mo, poems_pn], axis=0).reset_index(drop=True)

poems

0      \nThere is a thing in me that dreamed of trees...
1      \nShe sends me news of blue jays, frost,\nOf s...
2      \nShe steps into the dark swamp\nwhere the lon...
3      \nMy father, for example,\nwho was young once\...
4      \nClosing the book, I find I have left my head...
                             ...                        
221    \nThinking, tangling shadows in the deep solit...
222    \nYou are the daughter of the sea, oregano's f...
223    \nWhen I cannot look at your face\nI look at y...
224    \n\nWhen your hands leap\ntowards mine, love,\...
225    \n\nTake bread away from me, if you wish,\ntak...
Name: text, Length: 226, dtype: object

In [6]:
# lower all words:
corpus = []
for row in poems:
    #print(row)
    line = row.lower().split('\n')
    corpus.append(line)

In [ ]:
corpus

In [ ]:
corpus[3][38]

In [7]:
for i in range(0, len(corpus)):
    for j in range(0, len(corpus[i])):
        if '\xa0' in corpus[i][j]:
            #print(corpus[i][j])
            corpus[i][j]=''
            corpus[i][j+1]=''
        elif '<' in corpus[i][j]:
            corpus[i][j]=''
        elif 'neruda' in corpus[i][j]:
            corpus[i][j]=''

    

In [ ]:
corpus

In [ ]:
#corpus_cleaned = [item for sublist in corpus for item in sublist]
#corpus_cleaned = [item for item in corpus_cleaned if item.strip()]
#corpus_cleaned

In [8]:
# Get rid of the empty items of the lists and remove special characters: 
corpus_cleaned=[]
#
for i in range(0, len(corpus)):
    #corpus[i].pop(0)
    for j in range(0, len(corpus[i])):
        non_num = re.findall("([a-z]+)", corpus[i][j])
        non_num = ' '.join(non_num)
        corpus_cleaned.append(non_num)       
       
    
corpus_cleaned = [item for item in corpus_cleaned if item.strip()]

In [ ]:
corpus_cleaned

In [9]:
pd.DataFrame(corpus_cleaned).to_csv('mo_pn_corpus.csv', index=False)

In [10]:
#Tokenize all words of the poems: Give them numbers

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus_cleaned)
total_words = len(tokenizer.word_index) + 1 ## Add one to the length of the number of words to indicate, that we 
                                            ## have an out-of-vocabulary token
word_index = tokenizer.word_index

In [ ]:
print(total_words)
print(word_index)

In [11]:
import pickle

# open a file, where you ant to store the data
file = open('mo_pn_word_index.p', 'wb')

# dump information to that file
pickle.dump(word_index, file)

# close the file
file.close()

In [12]:
input_sequences = []

for line in corpus_cleaned:
    token_list = tokenizer.texts_to_sequences([line])[0]## Creates a sequence of numbers for every line
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]     ## This creates for each line a series of sequences with
        print(n_gram_sequence)                                       ## only some of the words: word 1 and 2 for the first
                                               ## sequence, word 1, 2 and 3 for the next and so on
        input_sequences.append(n_gram_sequence)

[34, 14]
[34, 14, 5]
[34, 14, 5, 238]
[34, 14, 5, 238, 7]
[34, 14, 5, 238, 7, 18]
[34, 14, 5, 238, 7, 18, 12]
[34, 14, 5, 238, 7, 18, 12, 2528]
[34, 14, 5, 238, 7, 18, 12, 2528, 3]
[34, 14, 5, 238, 7, 18, 12, 2528, 3, 144]
[5, 1204]
[5, 1204, 151]
[5, 1204, 151, 204]
[5, 1204, 151, 204, 119]
[5, 1204, 151, 204, 119, 2]
[5, 1204, 151, 204, 119, 2, 1205]
[5, 1204, 151, 204, 119, 2, 1205, 1625]
[5, 138]
[5, 138, 122]
[5, 138, 122, 17]
[5, 138, 122, 17, 123]
[5, 138, 122, 17, 123, 1626]
[5, 138, 122, 17, 123, 1626, 1206]
[5, 138]
[5, 138, 122]
[5, 138, 122, 17]
[5, 138, 122, 17, 2529]
[5, 138, 122, 17, 2529, 1207]
[5, 138, 122, 17, 2529, 1207, 1627]
[4, 115]
[4, 115, 44]
[4, 115, 44, 104]
[4, 115, 44, 104, 4]
[4, 115, 44, 104, 4, 212]
[4, 115, 44, 104, 4, 212, 2]
[4, 115, 44, 104, 4, 212, 2, 104]
[4, 115, 44, 104, 4, 212, 2, 104, 6]
[4, 115, 44, 104, 4, 212, 2, 104, 6, 2530]
[13, 73]
[13, 73, 2531]
[13, 73, 2531, 2]
[13, 73, 2531, 2, 161]
[13, 73, 2531, 2, 161, 20]
[13, 73, 2531, 2, 161, 2

[2, 102, 36, 28, 321, 22]
[2, 102, 36, 28, 321, 22, 636]
[2, 102, 36, 28, 321, 22, 636, 27]
[2, 102, 36, 28, 321, 22, 636, 27, 2682]
[2, 102, 36, 28, 321, 22, 636, 27, 2682, 2683]
[1245, 42]
[1245, 42, 1]
[1245, 42, 1, 1678]
[1245, 42, 1, 1678, 1679]
[1245, 42, 1, 1678, 1679, 1]
[1245, 42, 1, 1678, 1679, 1, 393]
[1245, 42, 1, 1678, 1679, 1, 393, 435]
[1245, 42, 1, 1678, 1679, 1, 393, 435, 2]
[1245, 42, 1, 1678, 1679, 1, 393, 435, 2, 28]
[1245, 42, 1, 1678, 1679, 1, 393, 435, 2, 28, 1]
[1245, 42, 1, 1678, 1679, 1, 393, 435, 2, 28, 1, 1680]
[1245, 42, 1, 1678, 1679, 1, 393, 435, 2, 28, 1, 1680, 329]
[9, 30]
[9, 30, 16]
[9, 30, 16, 5]
[9, 30, 16, 5, 1681]
[3, 138]
[3, 138, 274]
[2684, 3]
[2684, 3, 274]
[281, 8]
[281, 8, 1246]
[281, 8, 1246, 37]
[281, 8, 1246, 37, 40]
[281, 8, 1246, 37, 40, 165]
[281, 8, 1246, 37, 40, 165, 26]
[281, 8, 1246, 37, 40, 165, 26, 985]
[281, 8, 1246, 37, 40, 165, 26, 985, 42]
[281, 8, 1246, 37, 40, 165, 26, 985, 42, 208]
[1, 2685]
[1, 2685, 1]
[1, 2685, 1, 2686]

[2, 563, 1269, 37, 3, 9]
[2, 563, 1269, 37, 3, 9, 1014]
[2, 563, 1269, 37, 3, 9, 1014, 6]
[1015, 59]
[1015, 59, 1]
[1015, 59, 1, 1733]
[1015, 59, 1, 1733, 6]
[1015, 59, 1, 1733, 6, 646]
[1015, 59, 1, 1733, 6, 646, 7]
[32, 58]
[2, 140]
[2, 140, 140]
[2, 140, 140, 140]
[2, 140, 140, 140, 4]
[2, 140, 140, 140, 4, 117]
[2, 140, 140, 140, 4, 117, 114]
[2, 140, 140, 140, 4, 117, 114, 138]
[2, 140, 140, 140, 4, 117, 114, 138, 223]
[647, 1734]
[647, 1734, 57]
[647, 1734, 57, 2]
[647, 1734, 57, 2, 2789]
[647, 1734, 57, 2, 2789, 294]
[647, 1734, 57, 2, 2789, 294, 100]
[17, 1]
[17, 1, 2790]
[17, 1, 2790, 1735]
[17, 1, 2790, 1735, 12]
[17, 1, 2790, 1735, 12, 14]
[1736, 248]
[1736, 248, 67]
[2, 1016]
[61, 31]
[61, 31, 98]
[61, 31, 98, 43]
[61, 31, 98, 43, 2791]
[61, 31, 98, 43, 2791, 104]
[176, 20]
[176, 20, 77]
[176, 20, 77, 2792]
[176, 20, 77, 2792, 58]
[31, 76]
[31, 76, 53]
[31, 76, 53, 9]
[72, 1]
[72, 1, 638]
[72, 1, 638, 726]
[62, 2]
[62, 2, 60]
[59, 1]
[59, 1, 2793]
[59, 1, 2793, 95]
[1017, 2

[69, 28, 1042]
[69, 28, 1042, 4]
[69, 28, 1042, 4, 1274]
[69, 28, 1042, 4, 1274, 120]
[69, 28, 1042, 4, 1274, 120, 67]
[69, 28, 1042, 4, 1274, 120, 67, 227]
[4, 109]
[4, 109, 343]
[4, 109, 343, 5]
[4, 109, 343, 5, 2911]
[4, 109, 343, 5, 2911, 22]
[4, 109, 343, 5, 2911, 22, 2912]
[4, 109, 343, 5, 2911, 22, 2912, 3]
[4, 109, 343, 5, 2911, 22, 2912, 3, 262]
[4, 109, 343, 5, 2911, 22, 2912, 3, 262, 407]
[4, 109, 343, 5, 2911, 22, 2912, 3, 262, 407, 2]
[4, 109, 343, 5, 2911, 22, 2912, 3, 262, 407, 2, 2913]
[4, 109, 343, 5, 2911, 22, 2912, 3, 262, 407, 2, 2913, 229]
[833, 17]
[833, 17, 41]
[833, 17, 41, 2914]
[833, 17, 41, 2914, 1299]
[833, 17, 41, 2914, 1299, 17]
[833, 17, 41, 2914, 1299, 17, 41]
[833, 17, 41, 2914, 1299, 17, 41, 378]
[833, 17, 41, 2914, 1299, 17, 41, 378, 2915]
[833, 17, 41, 2914, 1299, 17, 41, 378, 2915, 298]
[20, 51]
[20, 51, 124]
[20, 51, 124, 49]
[20, 51, 124, 49, 8]
[20, 51, 124, 49, 8, 1805]
[20, 51, 124, 49, 8, 1805, 6]
[20, 51, 124, 49, 8, 1805, 6, 255]
[20, 51, 12

[124, 157, 464, 157, 1, 333, 8, 117, 49]
[124, 157, 464, 157, 1, 333, 8, 117, 49, 111]
[124, 157, 464, 157, 1, 333, 8, 117, 49, 111, 565]
[2, 4]
[2, 4, 115]
[2, 4, 115, 16]
[2, 4, 115, 16, 6]
[2, 4, 115, 16, 6, 44]
[2, 4, 115, 16, 6, 44, 356]
[2, 4, 115, 16, 6, 44, 356, 138]
[2, 4, 115, 16, 6, 44, 356, 138, 1850]
[2, 4, 115, 16, 6, 44, 356, 138, 1850, 34]
[2, 4, 115, 16, 6, 44, 356, 138, 1850, 34, 468]
[2, 4, 115, 16, 6, 44, 356, 138, 1850, 34, 468, 35]
[323, 280]
[323, 280, 128]
[323, 280, 128, 1]
[323, 280, 128, 1, 157]
[323, 280, 128, 1, 157, 14]
[323, 280, 128, 1, 157, 14, 1851]
[323, 280, 128, 1, 157, 14, 1851, 2]
[323, 280, 128, 1, 157, 14, 1851, 2, 1851]
[323, 280, 128, 1, 157, 14, 1851, 2, 1851, 37]
[323, 280, 128, 1, 157, 14, 1851, 2, 1851, 37, 3]
[323, 280, 128, 1, 157, 14, 1851, 2, 1851, 37, 3, 1]
[323, 280, 128, 1, 157, 14, 1851, 2, 1851, 37, 3, 1, 152]
[2, 664]
[2, 664, 64]
[2, 664, 64, 205]
[2, 664, 64, 205, 197]
[2, 664, 64, 205, 197, 715]
[2, 664, 64, 205, 197, 715, 353

...

[2, 2096, 1, 168, 605]
[2, 1]
[2, 1, 101]
[2, 1, 101, 670]
[2, 1, 101, 670, 5159]
[2, 1, 101, 670, 5159, 601]
[1, 1614]
[1, 1614, 382]
[1, 1614, 382, 35]
[1, 1614, 382, 35, 272]
[1, 1614, 382, 35, 272, 16]
[1, 1614, 382, 35, 272, 16, 5]
[1, 1614, 382, 35, 272, 16, 5, 553]
[405, 648]
[405, 648, 192]
[405, 648, 192, 2487]
[405, 648, 192, 2487, 19]
[405, 648, 192, 2487, 19, 1]
[405, 648, 192, 2487, 19, 1, 52]
[405, 5160]
[405, 5160, 450]
[405, 5160, 450, 95]
[405, 5160, 450, 95, 422]
[405, 5160, 450, 95, 422, 109]
[405, 5160, 450, 95, 422, 109, 82]
[405, 5160, 450, 95, 422, 109, 82, 54]
[2488, 91]
[2488, 91, 1028]
[2488, 91, 1028, 620]
[2488, 91, 1028, 620, 19]
[2488, 91, 1028, 620, 19, 620]
[875, 19]
[875, 19, 875]
[875, 19, 875, 7]
[875, 19, 875, 7, 1]
[875, 19, 875, 7, 1, 5161]
[875, 19, 875, 7, 1, 5161, 70]
[875, 19, 875, 7, 1, 5161, 70, 3]
[875, 19, 875, 7, 1, 5161, 70, 3, 1]
[875, 19, 875, 7, 1, 5161, 70, 3, 1, 1217]
[13, 5]
[13, 5, 5162]
[13, 5, 5162, 143]
[13, 5, 5162, 143, 3]
[13

[1, 5324, 266, 6]
[1, 5324, 266, 6, 950]
[2162, 2498]
[2162, 2498, 537]
[2162, 2498, 537, 2]
[2162, 2498, 537, 2, 458]
[1, 680]
[1, 680, 24]
[1, 680, 24, 5]
[1, 680, 24, 5, 5325]
[1, 680, 24, 5, 5325, 3]
[1, 680, 24, 5, 5325, 3, 668]
[28, 1]
[28, 1, 410]
[28, 1, 410, 743]
[28, 1, 410, 743, 6]
[28, 1, 410, 743, 6, 492]
[2, 47]
[2, 47, 684]
[2, 47, 684, 6]
[2, 47, 684, 6, 18]
[47, 5326]
[47, 5326, 7]
[47, 5326, 7, 1]
[47, 5326, 7, 1, 260]
[39, 83]
[39, 83, 169]
[444, 5327]
[444, 5327, 27]
[444, 5327, 27, 105]
[7, 1]
[7, 1, 203]
[7, 1, 203, 2]
[7, 1, 203, 2, 380]
[7, 1, 203, 2, 380, 3]
[7, 1, 203, 2, 380, 3, 1427]
[36, 312]
[36, 312, 47]
[36, 312, 47, 24]
[36, 312, 47, 24, 10]
[36, 312, 47, 24, 10, 815]
[36, 312, 47, 24, 10, 815, 22]
[36, 312, 47, 24, 10, 815, 22, 5328]
[22, 572]
[22, 572, 9]
[22, 572, 9, 24]
[22, 572, 9, 24, 604]
[22, 572, 9, 24, 604, 105]
[22, 572, 9, 24, 604, 105, 22]
[22, 572, 9, 24, 604, 105, 22, 21]
[1, 26]
[1, 26, 64]
[1, 26, 64, 133]
[1, 26, 64, 133, 21]
[1, 26, 6

[2, 4, 548, 440, 42]
[2, 4, 548, 440, 42, 1]
[2, 4, 548, 440, 42, 1, 877]
[2, 4, 548, 440, 42, 1, 877, 1551]
[148, 61]
[148, 61, 8]
[148, 61, 8, 63]
[148, 61, 8, 63, 6]
[148, 61, 8, 63, 6, 316]
[148, 61, 8, 63, 6, 316, 5]
[148, 61, 8, 63, 6, 316, 5, 641]
[148, 61, 8, 63, 6, 316, 5, 641, 5]
[148, 61, 8, 63, 6, 316, 5, 641, 5, 5513]
[148, 61, 8, 63, 6, 316, 5, 641, 5, 5513, 5]
[148, 61, 8, 63, 6, 316, 5, 641, 5, 5513, 5, 338]
[3, 11]
[3, 11, 1570]
[3, 11, 1570, 2]
[3, 11, 1570, 2, 5514]
[3, 11, 1570, 2, 5514, 38]
[3, 11, 1570, 2, 5514, 38, 10]
[3, 11, 1570, 2, 5514, 38, 10, 200]
[4, 115]
[4, 115, 1906]
[4, 115, 1906, 11]
[4, 115, 1906, 11, 142]
[4, 115, 1906, 11, 142, 156]
[4, 115, 1906, 11, 142, 156, 5515]
[4, 115, 1906, 11, 142, 156, 5515, 11]
[4, 115, 1906, 11, 142, 156, 5515, 11, 2366]
[4, 115, 1906, 11, 142, 156, 5515, 11, 2366, 3]
[4, 115, 1906, 11, 142, 156, 5515, 11, 2366, 3, 5516]
[11, 5517]
[11, 5517, 1224]
[11, 5517, 1224, 3]
[11, 5517, 1224, 3, 159]
[11, 5517, 1224, 3, 159, 2

[1079, 37]
[1079, 37, 5610]
[1079, 37, 5610, 214]
[1079, 37, 5610, 214, 4]
[1079, 37, 5610, 214, 4, 457]
[1079, 37, 5610, 214, 4, 457, 3]
[1079, 37, 5610, 214, 4, 457, 3, 1]
[1079, 37, 5610, 214, 4, 457, 3, 1, 178]
[4, 98]
[4, 98, 43]
[4, 98, 43, 79]
[4, 98, 43, 79, 6]
[4, 98, 43, 79, 6, 85]
[4, 98, 43, 79, 6, 85, 19]
[4, 98, 43, 79, 6, 85, 19, 170]
[4, 98, 43, 79, 6, 85, 19, 170, 5]
[4, 98, 43, 79, 6, 85, 19, 170, 5, 831]
[4, 98, 43, 79, 6, 85, 19, 170, 5, 831, 7]
[4, 98, 43, 79, 6, 85, 19, 170, 5, 831, 7, 1]
[4, 98, 43, 79, 6, 85, 19, 170, 5, 831, 7, 1, 74]
[5611, 603]
[5611, 603, 37]
[5611, 603, 37, 1893]
[5611, 603, 37, 1893, 13]
[5611, 603, 37, 1893, 13, 245]
[203, 19]
[203, 19, 62]
[203, 19, 62, 38]
[203, 19, 62, 38, 1]
[203, 19, 62, 38, 1, 600]
[203, 19, 62, 38, 1, 600, 2399]
[203, 19, 62, 38, 1, 600, 2399, 3]
[203, 19, 62, 38, 1, 600, 2399, 3, 1]
[203, 19, 62, 38, 1, 600, 2399, 3, 1, 52]
[642, 7]
[642, 7, 2]
[642, 7, 2, 471]
[642, 7, 2, 471, 2047]
[642, 7, 2, 471, 2047, 123]
[6

[4, 117, 4, 5710, 175, 5711, 3, 10, 55, 169]
[4, 117, 4, 5710, 175, 5711, 3, 10, 55, 169, 8]
[10, 55]
[10, 55, 169]
[10, 55, 169, 664]
[10, 55, 169, 664, 10]
[10, 55, 169, 664, 10, 594]
[10, 55, 169, 664, 10, 594, 55]
[1, 934]
[1, 934, 2405]
[1, 934, 2405, 1]
[1, 934, 2405, 1, 68]
[1, 934, 2405, 1, 68, 120]
[1, 934, 2405, 1, 68, 120, 1]
[1, 934, 2405, 1, 68, 120, 1, 435]
[980, 1483]
[980, 1483, 909]
[980, 1483, 909, 7]
[980, 1483, 909, 7, 1]
[980, 1483, 909, 7, 1, 68]
[980, 1483, 909, 7, 1, 68, 995]
[1, 613]
[1, 613, 2525]
[1, 613, 2525, 1]
[1, 613, 2525, 1, 934]
[1, 613, 2525, 1, 934, 1]
[1, 613, 2525, 1, 934, 1, 502]
[1, 613, 2525, 1, 934, 1, 502, 3]
[1, 613, 2525, 1, 934, 1, 502, 3, 1]
[1, 613, 2525, 1, 934, 1, 502, 3, 1, 68]
[1541, 1195]
[1541, 1195, 603]
[1541, 1195, 603, 616]
[1541, 1195, 603, 616, 1]
[1541, 1195, 603, 616, 1, 152]
[8, 252]
[8, 252, 40]
[8, 252, 40, 63]
[8, 252, 40, 63, 8]
[8, 252, 40, 63, 8, 34]
[8, 252, 40, 63, 8, 34, 40]
[8, 252, 40, 63, 8, 34, 40, 935]
[8, 25

In [ ]:
len(input_sequences)

In [13]:
#Pad the input_sequences:

max_sequence_len = max([len(x) for x in input_sequences])   ## Find the maximum length of the sequences 

padding_style='pre'

input_sequences = np.array(pad_sequences(input_sequences,         ## Pad everything with zeros up to the maximum length sentence
                                         maxlen=max_sequence_len, 
                                         padding=padding_style))



In [14]:
## Split into y and X; y is always the last number in the list

xs = input_sequences[:, :-1]
labels = input_sequences[:, -1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words) ## so we can one-hot-encode the labels

In [ ]:
ys

In [15]:
model = Sequential()
model.add(Embedding(total_words, 180, input_length = max_sequence_len-1))
model.add(Bidirectional(LSTM(200)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100
1017/1017 [==============================] - 134s 124ms/step - loss: 6.5870 - accuracy: 0.1047
Epoch 2/100
1017/1017 [==============================] - 129s 126ms/step - loss: 5.9987 - accuracy: 0.1339
Epoch 3/100
1017/1017 [==============================] - 135s 133ms/step - loss: 5.3322 - accuracy: 0.1534
Epoch 4/100
1017/1017 [==============================] - 133s 131ms/step - loss: 4.4669 - accuracy: 0.1979
Epoch 5/100
1017/1017 [==============================] - 127s 125ms/step - loss: 3.9783 - accuracy: 0.2389
Epoch 6/100
1017/1017 [==============================] - 124s 122ms/step - loss: 3.6455 - accuracy: 0.2757
Epoch 7/100
1017/1017 [==============================] - 124s 122ms/step - loss: 3.4091 - accuracy: 0.3007
Epoch 8/100
1017/1017 [==============================] - 122s 120ms/step - loss: 3.2064 - accuracy: 0.3302
Epoch 9/100
1017/1017 [==============================] - 118s 116ms/step - loss: 3.0384 - accuracy: 0.3521
Epoch 10/100
1017/1017 [=============

Epoch 77/100
1017/1017 [==============================] - 219s 216ms/step - loss: 2.1440 - accuracy: 0.5318
Epoch 78/100
1017/1017 [==============================] - 218s 215ms/step - loss: 2.1018 - accuracy: 0.5390
Epoch 79/100
1017/1017 [==============================] - 195s 192ms/step - loss: 2.1219 - accuracy: 0.5375
Epoch 80/100
1017/1017 [==============================] - 201s 198ms/step - loss: 2.1204 - accuracy: 0.5365
Epoch 81/100
1017/1017 [==============================] - 162s 159ms/step - loss: 2.1459 - accuracy: 0.5331
Epoch 82/100
1017/1017 [==============================] - 153s 151ms/step - loss: 2.1133 - accuracy: 0.5378
Epoch 83/100
1017/1017 [==============================] - 158s 155ms/step - loss: 2.0861 - accuracy: 0.5418
Epoch 84/100
1017/1017 [==============================] - 150s 147ms/step - loss: 2.0568 - accuracy: 0.5464
Epoch 85/100
1017/1017 [==============================] - 163s 160ms/step - loss: 2.0921 - accuracy: 0.5447
Epoch 86/100
1017/1017 [====

In [16]:
model.save('models/mo_pn_model')

INFO:tensorflow:Assets written to: models/mo_pn_model\assets


INFO:tensorflow:Assets written to: models/mo_pn_model\assets


In [17]:
import pickle

# open a file, where you ant to store the data
file = open('history_mo_pn.p', 'wb')

# dump information to that file
pickle.dump(history.history, file)

# close the file
file.close()

In [19]:
history['accuracy']

TypeError: 'History' object is not subscriptable

In [18]:
import matplotlib.pyplot as plt

plt.plot(history['accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

TypeError: 'History' object is not subscriptable

In [ ]:
seed_text = "honeysuckle"
next_words = 10


output_index = [0,0,0]
output_word = [0,0,0]

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    
        # if-clauses I added, because otherwise the model sometimes gives the same word several times in a row:
    if predicted[0] == output_index[-1]:
        predicted = np.argsort(model.predict(token_list), axis=-1)[0][-2]
    
    # also this. Otherwise it sometimes gives a pair of two words several times in a row:
    if (predicted[0] == output_index[-2]) & (output_word[-1] == output_word[-3]):
        predicted = np.argsort(model.predict(token_list), axis=-1)[0][-2]
    
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word.append(word)
            output_index.append(index)
            break
    seed_text += " " + output_word[-1]
print(seed_text)

In [ ]:

# check if last word is one of the non_words and if yes, chop it, than again test until we do not have a non_word
# maybe create your own list

# give an input word -> choose 10 or so words from the vocabulary we have here and choose the best fit. use this as seed_text.
# use a random number of 6-10 words or so as next_words

#use the next best fitting word and so on, don't know if it will  work fine

In [ ]:
#no_ending_word=['with', 'which', 'the', 'and', 'or', 'a', 'an', 'of', 'your', 'our', 'my']

In [ ]:
input_word = input("Enter a noun: ")

In [ ]:
#!pip install spacy --user

In [ ]:
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0-py3-none-any.whl

In [ ]:
import spacy
spacy.cli.download("en_core_web_md")

import random

In [ ]:
nlp = spacy.load('en_core_web_md')
topic = nlp(input_word)

#Create a list for the similarities:
sim_list = []
rand_word_list=[]

#choose 10 or so random words from word_index of data:
for i in range(0, 20):
    rand_word = random.choice(list(word_index.keys()))
    #print(rand_word)
    #find word in spacy
    rand_word_spacy = nlp(rand_word)
    rand_word_list.append(rand_word)

    #Compute similarity for the word:
    similarity = (topic.similarity(rand_word_spacy))
    sim_list.append(similarity)
    
#saves similarity to DataFrame
df_1 = pd.DataFrame({'similarity' : sim_list, 'word' : rand_word_list })
df_1.sort_values(by='similarity', inplace=True, ascending=False)


In [ ]:
#df_1 = df_1.reset_index()
#art=random.choice(['a', 'the'])
#words_to_use=[df_1['word'][0],df_1['word'][1], df_1['word'][2], 'the ' + input_word]

words_to_use=['the ' + input_word]

for i in range(0,4):
    words_to_use.append(df_1['word'][i])


In [ ]:
words_to_use

In [ ]:
poem = []
choices = random.sample(words_to_use, 3)
for item in choices: 
    seed_text = item
    print('this is the seed: ', seed_text)
    next_words = random.choice([5,6,7,8,9])
    

    output_index = [0,0,0]
    output_word = [0,0,0]
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model_mo.predict(token_list), axis=-1)
        
            # if-clauses I added, because otherwise the model sometimes gives the same word several times in a row:
        if predicted[0] == output_index[-1]:
            predicted = np.argsort(model_mo.predict(token_list), axis=-1)[0][-2]
        
        # also this. Otherwise it sometimes gives a pair of two words several times in a row:
        #if (predicted[0] == output_index[-2]) & (output_word[-1] == output_word[-3]):
        #    predicted = np.argsort(model_mo.predict(token_list), axis=-1)[0][-2]
        
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word.append(word)
                output_index.append(index)
                break
        seed_text += " " + output_word[-1]
    
    poem.append(seed_text)
print(seed_text)

In [ ]:
poem